In [216]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter

In [2]:
# nltk.corpus.gutenberg.fileids()
# total_corpus = []
# for file in nltk.corpus.gutenberg.fileids():
#     with open((nltk.corpus.gutenberg.words(file).fileid.path), 'r') as f:
#         for line in f.readlines():
#             try:
#                 total_corpus.append(line.split())
#             except:
#                 print ("hi")
#                 print (line)

# with open(emma.fileid.path) as f:
#     for line in f.readlines():
#         print (line)

# emma = nltk.corpus.gutenberg.words('austen-emma.txt')
# vocab = list(set(emma))
# len(vocab)

In [3]:
def extract_patterns_in_words(patterns,pattern_counter,word1,word2,max_len):
    i = 1
    while(word1[:i] == word2[:i]):
        i = i + 1
    if i != 1 and i > max(len(word1[i-1:]), len(word2[i-1:])) < max_len:
        pattern_counter[("suffix",word1[i-1:], word2[i-1:])] += 1
        if ("suffix",word1[i-1:], word2[i-1:]) in patterns:
            patterns[("suffix",word1[i-1:], word2[i-1:])].append((word1, word2))
        else:
            patterns[("suffix",word1[i-1:], word2[i-1:])] = [(word1, word2)]
#         patterns[("suffix",word1[i-1:], word2[i-1:], word1, word2)] += 1
    i = 1
    while(word1[-i:] == word2[-i:]):
        i = i + 1
    if i != 1 and max(len(word1[:-i+1]), len(word2[:-i+1])) < max_len:
        pattern_counter[("prefix",word1[:-i+1], word2[:-i+1])] += 1
        if ("prefix",word1[:-i+1], word2[:-i+1]) in patterns:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])].append((word1, word2))
        else:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])] = [(word1, word2)]
#         patterns[("prefix",word1[:-i+1], word2[:-i+1], word1, word2)] += 1
    return patterns

In [4]:
def build_pattern_dict(vocab,max_len = 6):
    if os.path.exists('../data/patterns_'+ str(len(vocab))):
        patterns_file_r = open('../data/patterns_'+ str(len(vocab)), 'rb')
        pattern_counter_file_r = open('../data/patterns_counter_'+ str(len(vocab)), 'rb')
        patterns = pickle.load(patterns_file_r)
        pattern_counter = pickle.load(pattern_counter_file_r)
    else:
        patterns_file_w = open('../data/patterns_'+ str(len(vocab)),"wb" )
        pattern_counter_file_w = open('../data/patterns_counter_'+ str(len(vocab)),"wb" )
        patterns  = defaultdict(list)
#         print (patterns)
        pattern_counter = Counter()
        for word in vocab:
            for second_word in vocab:
                if word != second_word:
                    extract_patterns_in_words(patterns,pattern_counter,word,second_word,max_len)
        pickle.dump(patterns, patterns_file_w)
        patterns_file_w.close()
        pickle.dump(pattern_counter, pattern_counter_file_w)
        pattern_counter_file_w.close()
    return patterns, pattern_counter

In [5]:
def downsample_patterns():
    #Downsample to include only top 1000
    pattern_1000 = defaultdict(list)
    for pattern,items in patterns.items():
        shuffle(items)
        pattern_1000[pattern] = items[:1000]
    return pattern_1000

In [6]:
def pair_wise_similarity(word_pair1, word_pair2,annoy_index=None, topn = 10,):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False

def annoy_pair_wise_similarity(word_pair1, word_pair2,annoy_index, topn = 10):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn, indexer=annoy_index)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False

def get_similarity_rank(word_pair1, word_pair2, topn=500):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
#     print (word_pair2[1])
#     print (closest_n)
    for n,(word, cos_sim) in enumerate(closest_n):
        if word == word_pair2[1]:
            return (n, cos_sim)
    return (None, None)

In [7]:
def index_vector(word_vectors, dimensions=300):
    fname = '../data/annoy.index'
    # Persist index to disk
    if os.path.exists(fname):
        annoy_index = AnnoyIndexer()
        annoy_index.load(fname)
        annoy_index.model = word_vectors
    else:
        annoy_index = AnnoyIndexer(word_vectors, dimensions)
        annoy_index.save(fname)
    return annoy_index

In [8]:
def get_hit_rate(patterns, similarity_function):
    if os.path.exists('../data/hitrate_'+ str(len(word_vectors.vocab))):
        hit_rate_file_r = open('../data/hitrate_'+ str(len(word_vectors.vocab)), 'rb')
        hit_rates_rules = pickle.load(hit_rate_file_r)
        return hit_rates_rules
    else:
        hit_rate_file_w = open('../data/hitrate_'+ str(len(word_vectors.vocab)),"wb" )
        hit_rates_rules = {}
        for (pattern,support_set) in patterns.items():
            hit_rates_word_pair = {}
            for pair1 in support_set:
                hit_count = 0
                hit_pairs = set()
                for pair2 in support_set:
                    if pair1 != pair2 and similarity_function(pair1, pair2, 10):
                        hit_count += 1
                        hit_pairs.add(pair2)
                if len(support_set) ==1:
                    total = 1
                else:
                    total = len(support_set) - 1
                if hit_count != 0:
                    hit_rates_word_pair[pair1] =  hit_pairs
            if len(support_set) != 1 and hit_rates_word_pair:
                hit_rates_rules[pattern] = hit_rates_word_pair
        pickle.dump(hit_rates_rules, hit_rate_file_w)
        hit_rate_file_w.close()
        return hit_rates_rules

In [9]:
#pattern_counter.most_common()[:-20:-1]

In [10]:
%time word_vectors = KeyedVectors.load_word2vec_format('/home/raja/models/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=1000)

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 50.5 ms


In [11]:
%time patterns, pattern_counter = build_pattern_dict(word_vectors.vocab.keys())

CPU times: user 128 ms, sys: 8 ms, total: 136 ms
Wall time: 142 ms


In [12]:
pattern_counter.most_common(20)

[(('suffix', 's', ''), 69),
 (('suffix', '', 's'), 69),
 (('suffix', '', 'ed'), 16),
 (('suffix', 'ed', ''), 16),
 (('suffix', '', 'd'), 13),
 (('suffix', 'd', ''), 13),
 (('suffix', 'n', ''), 11),
 (('suffix', '', 'n'), 11),
 (('suffix', 'ing', ''), 10),
 (('suffix', '', 'ing'), 10),
 (('suffix', '', 'er'), 8),
 (('prefix', '#', ''), 8),
 (('suffix', 'ing', 'e'), 8),
 (('suffix', 'er', ''), 8),
 (('prefix', '', '#'), 8),
 (('suffix', 'e', 'ing'), 8),
 (('prefix', 't', 'T'), 7),
 (('prefix', 'T', 't'), 7),
 (('suffix', '', 't'), 7),
 (('suffix', 't', ''), 7)]

In [13]:
%time sampled_patterns = downsample_patterns()

CPU times: user 128 ms, sys: 8 ms, total: 136 ms
Wall time: 152 ms


In [14]:
# word_vectors_annoy = deepcopy(word_vectors)
# word_vectors.init_sims(replace=True)
# %time annoy_index = index_vector(word_vectors=word_vectors, dimensions=300)
# %time hit_rates = calculate_hit_rate(sampled_patterns, annoy_pair_wise_similarity, annoy_index)

In [15]:
#pair_wise_similarity(("asking", "banking"), ("ask", "bank"))

In [16]:
%time hit_rates = get_hit_rate(sampled_patterns, pair_wise_similarity)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 15.2 ms


In [17]:
# def temp(pattern, transformations):
# #     ll = [transformations[transformation] for transformation in transformations]
# #     print (ll)
# #     print (len(ll), len(transformations))
#     print (transformations)
#     prototypes = []
#     support_set = set(sampled_patterns[pattern])
# #     print (pattern)
# #     print (support_set)
#     while True:
#         explains_by_count = sorted(transformations.items(), key=lambda kv: -len(kv[1]))
#         print (explains_by_count)
#         best = explains_by_count[0]
#         if len(best[1]) >= 10: #The prototype explains more than 10 word pairs
#             prototypes.append((best[0][0], len(best[1]) / float(len(support_set))))
#         else:
#             break
#         del transformations[best[0]]
#         #Remove all explained pairs from the support set
#         support_set = support_set - best[1]
#         for k, v in transformations.items():
#             transformations[k] = transformations[k] - best[1]
#         explains_by_count.pop(0)

#         if not (len(support_set) >= 10 and len(explains_by_count) and explains_by_count[0] >= 10):
#             break
#     print (prototypes)

In [18]:
#hit_rates

In [19]:
cons = 4
def update_morpho_rules(patterns):
    global morphological_rules
    for pattern in patterns:
        transformations = patterns[pattern]
#         ll = [transformations[transformation] for transformation in transformations]
    #     print (ll)
    #     print (len(ll), len(transformations))
        support_set = set(sampled_patterns[pattern])
        while(True):
            transformations_by_count = sorted(transformations.items(), key=lambda kv: len(kv[1]), reverse=True)
            best = transformations_by_count[0]
    #         print (transformations_by_count)
    #         print (transformations)
            if len(best[1]) >= cons:
                morphological_rules[best[0]] = (pattern, len(best[1]) / float(len(support_set)),  best[1])
    #             directions.append(best)
                del transformations[best[0]]
            else:
                break

            #Remove all explained pairs from the support set
            #TODO: Remove best[0] from support set and transformations
            support_set = support_set - best[1]
            for k, v in transformations.items():
    #             print ("*"*50)
    #             print (transformations[k])
                transformations[k] = transformations[k] - best[1]
    #             print (transformations[k])
    #             print ("__"*50)

            transformations_by_count.pop(0)
            if not (len(support_set) >= cons and len(transformations_by_count) and len(transformations_by_count[0][1]) >= cons):
                break
    #     return directions
    #         print (best_transformation,best_support)


In [20]:
#%time patterns = calculate_hit_rate(sampled_patterns, pair_wise_similarity)
morphological_rules = {}
%time update_morpho_rules(hit_rates)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.41 ms


In [21]:
print (len(morphological_rules))

32


In [22]:
morphological_rules

{('####', '###'): (('prefix', '#', ''),
  0.875,
  {('###', '##'),
   ('###,###', '##,###'),
   ('##,###', '#,###'),
   ('##-#', '#-#'),
   ('##.#', '#.#'),
   ('##.##', '#.##'),
   ('##:##', '#:##')}),
 ('#-#', '##-#'): (('prefix', '', '#'),
  0.75,
  {('##', '###'),
   ('##,###', '###,###'),
   ('#,###', '##,###'),
   ('#.#', '##.#'),
   ('#.##', '##.##'),
   ('#:##', '##:##')}),
 ('According', 'according'): (('prefix', 'A', 'a'),
  0.8333333333333334,
  {('After', 'after'),
   ('All', 'all'),
   ('An', 'an'),
   ('As', 'as'),
   ('At', 'at')}),
 ('News', 'news'): (('prefix', 'N', 'n'),
  0.8,
  {('National', 'national'), ('New', 'new'), ('No', 'no'), ('Now', 'now')}),
 ('She', 'she'): (('prefix', 'S', 's'),
  0.8,
  {('School', 'school'), ('So', 'so'), ('Some', 'some'), ('State', 'state')}),
 ('This', 'this'): (('prefix', 'T', 't'),
  0.8571428571428571,
  {('That', 'that'),
   ('The', 'the'),
   ('There', 'there'),
   ('These', 'these'),
   ('They', 'they'),
   ('Two', 'two')}),
 (

In [315]:
for item in morphological_rules:
    print (item, morphological_rules[item])
    print ("*" * 50)

('the', 'The') (('prefix', 't', 'T'), 0.8571428571428571, {('this', 'This'), ('they', 'They'), ('that', 'That'), ('there', 'There'), ('these', 'These'), ('two', 'Two')})
**************************************************
('#-#', '##-#') (('prefix', '', '#'), 0.75, {('##', '###'), ('##,###', '###,###'), ('#,###', '##,###'), ('#.#', '##.#'), ('#:##', '##:##'), ('#.##', '##.##')})
**************************************************
('This', 'this') (('prefix', 'T', 't'), 0.8571428571428571, {('There', 'there'), ('The', 'the'), ('They', 'they'), ('Two', 'two'), ('That', 'that'), ('These', 'these')})
**************************************************
('with', 'With') (('prefix', 'w', 'W'), 0.8333333333333334, {('what', 'What'), ('we', 'We'), ('world', 'World'), ('while', 'While'), ('when', 'When')})
**************************************************
('know', 'known') (('suffix', '', 'n'), 0.45454545454545453, {('America', 'American'), ('take', 'taken'), ('give', 'given'), ('A', 'An'), ('see'

In [24]:
for rule in morphological_rules:
    print (rule)
    break

('the', 'The')


In [25]:
# if os.path.exists('../data/rules_'+ str(len(word_vectors.vocab))):
#     rule_file_r = open('../data/rules_'+ str(len(word_vectors.vocab)), 'rb')
#     morphological_rules = pickle.load(rule_file_r)
# else:
#     hit_rates = calculate_hit_rate(sampled_patterns, pair_wise_similarity)
#     hit_rate_file_w = open('../data/rules_'+ str(len(word_vectors.vocab)),"wb" )
#     pickle.dump(hit_rates, hit_rate_file_w)
#     hit_rate_file_w.close()
# # %time hit_rates = calculate_hit_rate(sampled_patterns, pair_wise_similarity)

In [307]:
G = nx.MultiDiGraph()

In [308]:
%time G.add_nodes_from(word_vectors.vocab.keys())

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 5.56 ms


In [309]:
for dw,support in morphological_rules.items():
    morp_rule, hit_rate,support_set = support
    (word1, word2) = dw
    for (word3, word4) in support_set:
        (rank,cos_sim) = get_similarity_rank((word1,word2),(word3,word4))
        if rank < 3 and cos_sim > 0.5:
            G.add_edge(word3,word4,dw=dw,cos=0.35,rank=1)
            if not G.has_edge(word3,word4,key=dw):
                G.add_edge(word3,word4,key=dw,cos=cos_sim,rank=rank)
        else:
#             print (rank,cos_sim, word3, word2, dw)
            pass

#         G.add_edge(word1, word2 )

In [310]:
for item in G['make']:
    print (item)
    d = G['make'][item]
    print(d)
print (G.out_edges('make',data=True))
print (G.out_edges('make',keys=True))

makes
{0: {'dw': ('need', 'needs'), 'cos': 0.35, 'rank': 1}, ('need', 'needs'): {'cos': 0.5872986912727356, 'rank': 1}}
making
{0: {'dw': ('give', 'giving'), 'cos': 0.35, 'rank': 1}, ('give', 'giving'): {'cos': 0.9000281095504761, 'rank': 0}}
[('make', 'makes', {'dw': ('need', 'needs'), 'cos': 0.35, 'rank': 1}), ('make', 'makes', {'cos': 0.5872986912727356, 'rank': 1}), ('make', 'making', {'dw': ('give', 'giving'), 'cos': 0.35, 'rank': 1}), ('make', 'making', {'cos': 0.9000281095504761, 'rank': 0})]
[('make', 'makes', 0), ('make', 'makes', ('need', 'needs')), ('make', 'making', 0), ('make', 'making', ('give', 'giving'))]


In [311]:
print (len(G.edges))
print (len(G.nodes))

348
1000


In [312]:
# for item in G.nodes:
#     if G.out_degree(item) > 1:
#         print (item, G.out_degree(item))

In [313]:
for node in list(G.nodes):
    for neighbor in list(G.neighbors(node)):
        if word_vectors.vocab[node].count > word_vectors.vocab[neighbor].count:
            if G.has_edge(node, neighbor):
                G.remove_edges_from(set(G.in_edges(neighbor,keys=True)) and set(G.out_edges(node,keys=True)))
            if G.number_of_edges(neighbor, node) > 1:
#                 print (list(G.in_edges(node,keys=True)))
                n_list = [(G[neighbor][node][item]['rank'], G[neighbor][node][item]['cos'], item) for item in (G[neighbor][node].keys())]
                min_rank_edge = min(n_list,key=itemgetter(0))
                max_cos_edge = max(n_list,key=itemgetter(1))
#                 print (list(G.in_edges(node,keys=True)))
                remove_edges = [x for x in list(G.in_edges(node,keys=True)) if x != (neighbor,node,min_rank_edge[2])]
#                 print (remove_edges)
                G.remove_edges_from(remove_edges)
                if G.number_of_edges(neighbor, node) > 1:
                    remove_edges = [x for x in list(G.in_edges(node,keys=True)) if x != (neighbor,node,max_cos_edge[2])]
                    G.remove_edges_from(remove_edges)
                    print (list(G.in_edges(node,keys=True)))
        else:
            if G.has_edge(neighbor, node):
                G.remove_edges_from(set(G.in_edges(node,keys=True)) and set(G.out_edges(neighbor,keys=True)))
            if G.number_of_edges(node, neighbor) > 1:
                n_list = [(G[node][neighbor][item]['rank'], G[node][neighbor][item]['cos'], item) for item in (G[node][neighbor].keys())]
                min_rank_edge = min(n_list,key=itemgetter(0))
                max_cos_edge = max(n_list,key=itemgetter(1))
                remove_edges = [x for x in list(G.in_edges(neighbor,keys=True)) if x != (node,neighbor,min_rank_edge[2])]
                G.remove_edges_from(remove_edges)
                if G.number_of_edges(node, neighbor) > 1:
                    remove_edges = [x for x in list(G.in_edges(neighbor,keys=True)) if x != (node,neighbor,max_cos_edge[2])]
                    G.remove_edges_from(remove_edges)

In [314]:
print (len(G.edges))
print (len(G.nodes))

76
1000
